<a target="_blank" href="https://colab.research.google.com/github/JLDC/Data-Science-Fundamentals/blob/master/notebooks/204-clustering.ipynb">
    <img src="https://i.ibb.co/2P3SLwK/colab.png"  style="padding-bottom:5px;" />Open this notebook in Google Colab
</a>

___

# Clustering
___

In this notebook, we will go over the concept of *clustering* in machine learning. In general, clustering belongs to a category of learning which we call **unsupervised learning**. Until now, the learning algorithms we have covered consisted in training a model using *labeled data*, i.e., we had a target label (e.g., the value of a house, the species of an iris, etc.) which we were trying to predict. This is what we call **supervised learning**. Indeed, you can think of our error function as a *supervisor* telling our model how well it is performing.

In unsupervised learning, our data is not labeled, so what is the goal? **Unsupervised learning** aims to learn about **relationships within the data**. As we will also see, unsupervised learning can be a useful step when we are doing supervised learning. In fact, you have already learned about an unsupervised learning algorithm: **principal component analysis**. Think about it, PCA knew nothing about the labels in our data, yet it was able to create variables that explain the variance in the data particularly well.

This might still sound a bit abstract for now, but there are a lot of use cases for unsupervised learning. For instance, if we want to reduce the dimensionality of our data to improve our training time in a supervised learning framework; or maybe we are making a recommendation system for an online shop and we want to find similar buying behaviors amongst customers; or perhaps we are trying to classify topics of news articles such that *similar* articles are grouped together. 

This notebook will focus on **clustering**, i.e., grouping *similar* points of data into clusters. In general, we differentiate between **hard clustering**, where we assign points to a specific cluster, and **soft clustering** where we assign a *probability* of a point being in each cluster.

___
## Centroid-based Clustering

**Centroid-based clustering** is one of the typical methods to build clusters. The general idea behind it is to associate each cluster with a **centre** and the simplest algorithm is known as **K-means clustering**. With K-means clustering, we choose a number of cluster centres $k$ and we assign the data points to the cluster with the *closest* center*.

### 🙀 🤯 Mathematical Formulation

Let $\mathbf{x}_1, \dots, \mathbf{x}_n$ be our data points. We want to pick $k$ cluster centres $\mathbf{c}_1, \dots, \mathbf{c}_k$ and assign each $\mathbf{x}_i$ to a cluster $\mathbf{z}_i \in \{1, \dots, k\}$ in order to minimize

$$f(\mathbf{c}_{1:k}, \mathbf{z}_{1:n}) = \sum_{i=1}^n d(\mathbf{x}_i, \mathbf{c}_{z_i}),$$

where $d(\cdot, \cdot)$ is a distance function.

Unfortunately, this type of minimization is an NP-hard problem in general (NP stands for *non-deterministic polynomial time* and NP-hard means that the problem is at least as hard as the hardest problems in non-deterministic polynomial time... in short: **this is a extremely difficult problem**).

K-means uses **coordinate descent** to find a **local minimum**, as you already know, finding local minima is generally much easier than finding global ones.

Let us start with an example of K-means on some data we already know fairly well, the iris flowers data set.

Assume the following situation: you know that there are three different type of flowers in your data, but given any data point, you don't know what kind of species it belongs to. You can think of this as having the iris data set but **without** the `species` column. For plotting purposes, we will keep it in, but you can consider it to unknown.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np

from sklearn.cluster import KMeans # K-means algorithm from scikit-learn

# Define the path where the data is stored
DATA_PATH = "https://raw.githubusercontent.com/JLDC/Data-Science-Fundamentals/master/data"

In [ ]:
# Read the iris data set
iris = pd.read_csv(f"{DATA_PATH}/iris.csv")

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))

ax.scatter(iris["sepal length (cm)"], iris["sepal width (cm)"], alpha=0.6)
    
ax.set_xlabel("Sepal length (cm)")
ax.set_ylabel("Sepal width (cm)")
ax.grid(linestyle="dashdot")

Look at the above picture, illustrating the single flowers based on their sepal length and sepal width. If this was our only information, it would be fairly difficult to classify the flowers into three clusters. We might be able to separate one cluster at the top left, but the two other clusters are harder to define. Luckily for us, K-means can help!

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=42)
X = np.asarray(iris[["sepal length (cm)", "sepal width (cm)"]])
kmeans.fit(X)

In [ ]:
# Use this custom function to plot your kmeans classification.
# No need to focus on the code, the important part is the resulting plot!
def plot_kmeans(kmeans, X, h=0.01):
    fig, ax = plt.subplots(figsize=(12, 8))
    # Plot the decision boundary
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    
    # Obtain labels for each point in mesh
    Z = kmeans.predict(np.c_[xx.ravel(), yy.ravel()])
    
    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    ax.imshow(
        Z, 
        interpolation="nearest", 
        extent=(xx.min(), xx.max(), yy.min(), yy.max()),
        cmap=mpl.colormaps["viridis"],
        aspect="auto",
        origin="lower",
        alpha=0.5
    )
    # Plot the scatter
    ax.scatter(X[:, 0], X[:, 1], color="black", edgecolor="white")
    # Plot the centroids
    centroids = kmeans.cluster_centers_
    ax.scatter(centroids[:, 0], centroids[:, 1], marker="X", s=180, linewidth=1, color="black", edgecolor="white")
    plt.show()

In [ ]:
plot_kmeans(kmeans, X)

The above plot shows how K-means clusters our data. Of course, K-means is somewhat random (i.e., it depends on where we set the cluster centroids to begin with), so using another `random_state` might yield different results.

K-means is an iterative procedure, meaning it rearranges the centroids iteration by iteration, the following GIF illustrates this process:

![](https://raw.githubusercontent.com/JLDC/Data-Science-Fundamentals/master/data/images/kmeans.gif)

We see that, in this case, the clusters do not change much past the first ten iterations. Furthermore, we see that **without any knowledge of the species**, and based uniquely on the sepal length and sepal width, K-means creates cluster that contain mostly observations of the same species. On the one hand, this is quite impressive, on the other hand, it's a clear indication that amongst a particular iris species, there are similarities in the sepal widths and sepal lengths!

___
## Soft Clustering


K-means is (typically) fast, but, in some situations, we might prefer probabilistic statements about clustering. Consider the results of the clustering for the iris data set, how confident do you feel about a point near the cluster's boundary belonging to a specific cluster vs a point far away?

Soft clustering, i.e., quantifying the probability that a data point belongs to a specific cluster, helps pin down this uncertainty.

#### ➡️ ✏️ <font style="color: green">**Question 1**</font>

Repeat the same analysis for the crabs dataset. Plot your results using the function provided above. What are your conclusions?

___
### Mixture models

Mixture models are a popular way to cluster data in a probabilistic way. Such a model takes the form

$$ p(\mathbf{x}) = \sum_{k=1}^K w_k p_k(\mathbf{x}).$$

The main idea behind this model is that the data $\mathbf{x}$ is generated by
+ selecting a mixture component $k$ with probability $w_k$,
+ generating $\mathbf{x}$ from the distribution associated with $p_k$.

Our goal is to learn $w_1, \dots w_K$ and the parameters associated with each component. Notice, that we assume that there are $K$ clusters.

#### 🙀 🤯 Example: Gaussian mixture model

Let $K=3$ and $\mathbf{x} \in \mathbb{R}^2$, with $p_k(\mathbf{x})$ a multivariate Gaussian with mean $\mathbf{\mu}_k$ and covariance $\mathbf{\Sigma}_k$.

We have 

$$p(\mathbf{x}) = \sum_{k=1}^K w_k \mathcal{N}\left(\mathbf{x}; \mathbf{\mu}_k, \mathbf{\Sigma}_k\right)$$

Say that we have $n=1\,000$ data points generated from the mixture model:

![ ](https://raw.githubusercontent.com/JLDC/Data-Science-Fundamentals/master/data/images/gaussianmixture1.png)

can we learn $\mathbf{w}_{1:K}$, $\mathbf{\mu}_{1:K}$, and $\mathbf{\Sigma}_{1:K}$?


But what if instead, our data looks like:

![](https://raw.githubusercontent.com/JLDC/Data-Science-Fundamentals/master/data/images/gaussianmixture2.png)

can we learn $\mathbf{w}_{1:K}$, $\mathbf{\mu}_{1:K}$, and $\mathbf{\Sigma}_{1:K}$ now?

Let's find out using scikit-learn's `GaussianMixture` function!

In [ ]:
from sklearn.mixture import GaussianMixture

In [ ]:
# Read the data points from the first plot
df = pd.read_csv(f"{DATA_PATH}/gaussian_mixture1.csv")
df # Display the contents

In [ ]:
# Create an instance of Gaussian mixture representation with 3 components
gaussian_mixture = GaussianMixture(n_components=3, random_state=52)

In [ ]:
# Fit it to the data points
gaussian_mixture.fit(df[["X", "Y"]])
# Predict the cluster using our Gaussian mixture
df["Cluster"] = gaussian_mixture.predict(df[["X", "Y"]])

In [ ]:
# Visualize the results

fig, ax = plt.subplots(figsize=(8, 8))

# Iterate over the predicted clusters
for cluster in df["Cluster"].unique():
    # Subset the dataframe
    dft = df[df["Cluster"] == cluster]
    
    # Make a scatterplot
    ax.scatter(dft["X"], dft["Y"], label=cluster, alpha=0.8)
    
# Prettify the plot
ax.legend(title="Predicted cluster")
ax.grid(linestyle="dashdot")

Compare this to the clusters of the picture above, they are just the same! Be careful, however, the colors do not need to match, the important part is that the **clusters** match, i.e., the singular points are correctly identified as belonging to the same cluster.

That's pretty neat, but does it also hold for the second image with more complicated data structure?

In [ ]:
# Read the data points from the second plot
df = pd.read_csv(f"{DATA_PATH}/gaussian_mixture2.csv")

In [ ]:
# Fit it to the data points
gaussian_mixture.fit(df[["X", "Y"]])
# Predict the cluster using our Gaussian mixture
df["Cluster"] = gaussian_mixture.predict(df[["X", "Y"]])

In [ ]:
# Visualize the results

fig, ax = plt.subplots(figsize=(8, 8))

# Iterate over the predicted clusters
for cluster in df["Cluster"].unique():
    # Subset the dataframe
    dft = df[df["Cluster"] == cluster]
    
    # Make a scatterplot
    ax.scatter(dft["X"], dft["Y"], label=cluster, alpha=0.8)
    
# Prettify the plot
ax.legend(title="Predicted cluster")
ax.grid(linestyle="dashdot")

Compare this to the second plot in the text above, the predicted clusters are very close to the true clusters! Even the two points belonging to the *blue* cluster on the far right in the center have been classified correctly. That's definitely something K-means would not have been able to do!

#### ➡️ ✏️<font color=green>**Question 2**</font>

Clustering is part of what we call unsupervised learning. Clusters can occur due to the way the *data generating process* works, but you might never be able to verify which cluster an observation belongs to. That is, in practice, you might never be able to assess how well your clustering works.

+ Does this mean unsupervised is less useful than supervised learning? Or more useful?
+ Can you think of real life examples where you would like to use unsupervised learning instead of supervised learning?

Discuss with your classmates.

#### ➡️ ✏️<font color=green>**Question 3**</font>

Create a Gaussian mixture model with 3 components and fit it on the iris data set. 

1. First, do so by using only the sepal length and sepal width as features.
2. Second, do so by using the petal length and petal width **additionally** (i.e., 4 features in total)

Plot the predicted clusters in a plot where the $x$-axis is the sepal length and the $y$-axis is the sepal width. What do you observe?

In [ ]:
# Enter your code here

#### ➡️ ✏️<font color=green>**Question 4**</font>

Repeat the same type of analysis with the crabs dataset. What do you find there? Can you say something about the number of clusters?

In [ ]:
# Entere your code here